In [ ]:
# импортируем библиотеки
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET

In [ ]:
# создание переменных
XML = "books.xml"
CSV = "books.csv"
URL = 'https://book24.ru'
HEADERS = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.64"
}

In [ ]:
# открытие ссылки
def get_html(url):
    r = requests.get(url, headers=HEADERS)
    return r

In [ ]:
# достаем контент
def get_content(html):
    soup = BeautifulSoup(html, "html.parser")
    items = soup.find_all("div", class_="embla-slide-product shelf-products__item")
    cards = []

    for item in items:
        cards.append(
            {
                "title": item.find('div', class_='product-card__content').get_text(strip=True),
                "author": item.find('div', class_="author-list product-card__authors-holder").get_text(strip=True)
            }
        )
    return cards

In [ ]:
html = get_html(URL)
print(get_content(html.text))

In [ ]:
# сохранение в csv файл
def save_csv(items, path):
    with open(path, "w", newline='') as file:
        writer = csv.writer(file, delimiter=";")
        writer.writerow(['Название книг', 'Автор'])
        for item in items:
            writer.writerow([item['title'], item['author']])

In [ ]:
# сохранение в xml файл
def save_xml(items, path):
    root = ET.Element('books')
    for i, item in enumerate(items, 1):
        person = ET.SubElement(root, 'person' + str(i))
        ET.SubElement(person, 'title').text = item['title']
        ET.SubElement(person, 'author').text = item['author']
    tree = ET.ElementTree(root)
    tree.write(path)

In [ ]:
# основная функция
def parser():
    html = get_html(URL)
    if html.status_code == 200:
        cards = []
        for page in range(1):
            if cards is not None:
                html = get_html(URL)
                cards.extend(get_content(html.text))
                save_csv(cards, CSV)
                save_xml(cards, XML)
    else:
        print("Error")

In [ ]:
# запуск программы
parser()